# Random Forest && XG Boost

In [192]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action="ignore")
path = './data/'
gearbox = 'GEARBOX_processed.csv'
generator_bearing = 'GENERATOR_BEARING_processed.csv'
generator = 'GENERATOR_processed.csv'
hydraulic_group = 'HYDRAULIC_GROUP_processed.csv'
transformer = 'TRANSFORMER_processed.csv'
all_signals = 'all_signals_processed.csv'

In [193]:
failures = pd.read_csv(path + "htw-failures-training.csv")
failures['Timestamp'] = pd.to_datetime(failures['Timestamp'])
failures['Timestamp'] = failures['Timestamp'].dt.round('10T')
failures['Label'] = 1

In [194]:
all_signals = pd.read_csv(path + all_signals)
all_signals['Timestamp'] = pd.to_datetime(all_signals['Timestamp'])
# features.columns

matched_rows = pd.merge(failures, all_signals, on=['Turbine_ID', 'Timestamp'])

# If you want to check the result
if not matched_rows.empty:
    print("There are matching rows based on 'Turbine_ID' and 'Timestamp'")
    print(len(matched_rows))
else:
    print("No matching rows found based on 'Turbine_ID' and 'Timestamp'")
# 3 missing failures are still not in the processed data

There are matching rows based on 'Turbine_ID' and 'Timestamp'
20


In [200]:
## Group by Turbine_ID and then normalize each features' data
from sklearn.preprocessing import StandardScaler
feature_columns = [col for col in all_signals if col not in ['Turbine_ID', 
                                                             'Timestamp', 
                                                             'Label_GENERATOR',
                                                             'Next_Default_Date_GENERATOR', 'Lead_Time_GENERATOR',
                                                             'Default_in_60_GENERATOR', 'Label_HYDRAULIC_GROUP',
                                                             'Next_Default_Date_HYDRAULIC_GROUP', 'Lead_Time_HYDRAULIC_GROUP',
                                                             'Default_in_60_HYDRAULIC_GROUP', 'Label_GENERATOR_BEARING',
                                                             'Next_Default_Date_GENERATOR_BEARING', 'Lead_Time_GENERATOR_BEARING',
                                                             'Default_in_60_GENERATOR_BEARING', 'Label_TRANSFORMER',
                                                             'Next_Default_Date_TRANSFORMER', 'Lead_Time_TRANSFORMER',
                                                             'Default_in_60_TRANSFORMER', 'Label_GEARBOX',
                                                             'Next_Default_Date_GEARBOX', 'Lead_Time_GEARBOX',
                                                             'Default_in_60_GEARBOX']]
def normalize_features(group):
    scaler = StandardScaler()
    group[feature_columns] = scaler.fit_transform(group[feature_columns])
    return group

normalized_signal = all_signals.groupby('Turbine_ID').apply(normalize_features).reset_index(drop=True)
normalized_signal


,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Gen_SlipRing_Temp_Avg,...,Lead_Time_GENERATOR_BEARING,Default_in_60_GENERATOR_BEARING,Label_TRANSFORMER,Next_Default_Date_TRANSFORMER,Lead_Time_TRANSFORMER,Default_in_60_TRANSFORMER,Label_GEARBOX,Next_Default_Date_GEARBOX,Lead_Time_GEARBOX,Default_in_60_GEARBOX
0,T01,2016-01-01 00:00:00+00:00,0.183783,0.517399,0.343201,-0.530101,-0.270840,-0.213316,-0.226285,-0.518503,...,89950,False,0.0,2017-08-11 13:10:00+00:00,588,False,0.0,2016-07-18 02:10:00+00:00,199,False
1,T01,2016-01-01 00:10:00+00:00,0.169904,-1.304604,-0.053454,3.336224,-0.270840,-0.213316,-0.191148,-0.518503,...,89949,False,0.0,2017-08-11 13:10:00+00:00,588,False,0.0,2016-07-18 02:10:00+00:00,199,False
2,T01,2016-01-01 00:20:00+00:00,0.362223,-1.100337,-0.412559,3.790873,-0.302901,-0.282257,-0.261422,-0.518503,...,89949,False,0.0,2017-08-11 13:10:00+00:00,588,False,0.0,2016-07-18 02:10:00+00:00,199,False
3,T01,2016-01-01 00:30:00+00:00,0.228317,0.495364,0.356088,-0.457647,-0.334963,-0.282257,-0.261422,-0.518503,...,89949,False,0.0,2017-08-11 13:10:00+00:00,588,False,0.0,2016-07-18 02:10:00+00:00,199,False
4,T01,2016-01-01 00:40:00+00:00,0.250126,0.514182,0.357043,-0.448590,-0.302901,-0.247786,-0.261422,-0.518503,...,89949,False,0.0,2017-08-11 13:10:00+00:00,588,False,0.0,2016-07-18 02:10:00+00:00,199,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434110,T11,2017-08-31 23:20:00+00:00,0.308787,0.457460,0.284569,-0.227676,-0.305246,-0.286253,-0.318863,0.317192,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
434111,T11,2017-08-31 23:30:00+00:00,0.025024,0.449680,0.221720,-0.616017,-0.262225,-0.242795,-0.231583,0.317192,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
434112,T11,2017-08-31 23:40:00+00:00,0.024017,0.443629,0.219989,-0.610560,-0.219203,-0.199337,-0.187943,0.317192,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
434113,T11,2017-08-31 23:50:00+00:00,0.052209,0.438097,0.222932,-0.596009,-0.176182,-0.155880,-0.144302,0.317192,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False


### Perform CUSUM (Abandoned)

In [168]:
# def calculate_deviation(df):
#     # Drop irrelevant column
#     df_filtered = df.drop(columns=['Timestamp', 'Label', 'Next_Default_Date', 'Lead_Time', 'Default_in_60'])
#     df_normal = df_filtered[df['Default_in_60'] == 0]
#     grouped_means = df_normal.groupby('Turbine_ID').mean().reset_index()
#     df_merged = pd.merge(df, grouped_means,
#                                  on='Turbine_ID', suffixes=('', '_Mean'))
#     # Calculate deviation for each feature
#     feature_columns = [col for col in df_filtered.columns if col not in ['Turbine_ID', 'Default_in_60']]
#     for col in feature_columns:
#         mean_col = f'{col}_Mean'  # Construct the name of the mean column
#         deviation_col = f'{col}_Dev' # Name for the new deviation column
#         df_merged[deviation_col] = df_merged[col] - df_merged[mean_col]
#     return df_merged

In [169]:
# def cusum(series):
#     cum_sum = [0] # Initialize with 0 or the series' first value as desired
#     for i in range(1, len(series)):
#         # For a basic CUSUM, we're just accumulating deviations
#         cum_sum.append(cum_sum[-1] + series.iloc[i])
#     return pd.Series(cum_sum, index=series.index)
# def apply_cusum_to_group(group):
#     deviation_columns = [col for col in group.columns if col not in ['Turbine_ID', 'Default_in_60']]
#     for col in deviation_columns:
#         group[col + '_CUSUM'] = cusum(group[col])
#     return group

In [170]:
# def combine_feature_label(df):
#     df_deviation = calculate_deviation(df)
#     columns_to_keep = ['Turbine_ID', 'Default_in_60']
#     columns_dev = [col for col in df_deviation.columns if '_Dev' in col]
#     columns_to_keep.extend(columns_dev)
#     df_deviation = df_deviation[columns_to_keep]
#     df_cusum = df_deviation.groupby('Turbine_ID', as_index=False).apply(apply_cusum_to_group)
#     columns_features = [col for col in df_cusum.columns if '_CUSUM' in col]
#     columns_to_keep2 =  ['Turbine_ID', 'Default_in_60']
#     columns_to_keep2.extend(columns_features)
#     df_cusum = df_cusum[columns_to_keep2]
#     return df_cusum

In [211]:
# gearbox_df = pd.read_csv(path + gearbox)
# gearbox_df['Timestamp'] = pd.to_datetime(gearbox_df['Timestamp'])
# gearbox_cusum = combine_feature_label(gearbox_df)
# gearbox_cusum.columns

### Resampling

In [212]:
# Apply SMOTE to my data
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

normalized_signal_sorted = normalized_signal.sort_values(by='Timestamp')
train_size = int(len(normalized_signal_sorted) * 0.8)
train_df = normalized_signal_sorted.iloc[:train_size]
test_df = normalized_signal_sorted.iloc[train_size:]
test_df

# Y_labels = ['Default_in_60_GENERATOR', 'Default_in_60_HYDRAULIC_GROUP', 'Default_in_60_GENERATOR_BEARING', 'Default_in_60_TRANSFORMER', 'Default_in_60_GEARBOX']
# def split_x_y(y_label, lead_time_label, df=normalized_signal):
#     X = df.drop(['Turbine_ID',
#                 'Timestamp',
#                 'Label_GENERATOR',
#                 'Next_Default_Date_GENERATOR', 'Lead_Time_GENERATOR',
#                 'Default_in_60_GENERATOR', 'Label_HYDRAULIC_GROUP',
#                 'Next_Default_Date_HYDRAULIC_GROUP', 'Lead_Time_HYDRAULIC_GROUP',
#                 'Default_in_60_HYDRAULIC_GROUP', 'Label_GENERATOR_BEARING',
#                 'Next_Default_Date_GENERATOR_BEARING', 'Lead_Time_GENERATOR_BEARING',
#                 'Default_in_60_GENERATOR_BEARING', 'Label_TRANSFORMER',
#                 'Next_Default_Date_TRANSFORMER', 'Lead_Time_TRANSFORMER',
#                 'Default_in_60_TRANSFORMER', 'Label_GEARBOX',
#                 'Next_Default_Date_GEARBOX', 'Lead_Time_GEARBOX',
#                 'Default_in_60_GEARBOX'], axis=1)
#     y = df[[y_label, lead_time_label]].astype(int)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#     smote = SMOTE(random_state=42)
#     # Resample the training data
#     y_train = y_train[y_label]
#     y_test_lead_time = y_test[lead_time_label]
#     y_test = y_test[y_label]
#     X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
#     return X_train_smote, X_test, y_train_smote, y_test, y_test_lead_time

# rf_model = {}
# for y in Y_labels:
#     X_train_smote, X_test, y_train_smote, y_test = split_x_y(y)
#     rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
#     rf_clf.fit(X_train_smote, y_train_smote)
#     y_predict = rf_clf.predict(X_test)
#     report = classification_report(y_test, y_predict)
#     print("For " + y)
#     print(report)
# X_train_smote, X_test, y_train_smote, y_test, y_test_lead_time = split_x_y('Default_in_60_HYDRAULIC_GROUP', 'Lead_Time_HYDRAULIC_GROUP')
# rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_clf.fit(X_train_smote, y_train_smote)
# y_predict = rf_clf.predict(X_test)
# report = classification_report(y_test, y_predict)
# print("For " + y)
# print(report)


,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Gen_SlipRing_Temp_Avg,...,Lead_Time_GENERATOR_BEARING,Default_in_60_GENERATOR_BEARING,Label_TRANSFORMER,Next_Default_Date_TRANSFORMER,Lead_Time_TRANSFORMER,Default_in_60_TRANSFORMER,Label_GEARBOX,Next_Default_Date_GEARBOX,Lead_Time_GEARBOX,Default_in_60_GEARBOX
155127,T06,2017-05-02 00:20:00+00:00,0.166058,0.507718,0.342649,-0.540645,-0.762760,-0.752021,-0.714676,-0.543135,...,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False
155128,T06,2017-05-02 00:30:00+00:00,0.167598,0.523127,0.343931,-0.549653,-0.610862,-0.604845,-0.599802,-0.543135,...,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False
69854,T01,2017-05-02 00:30:00+00:00,0.169752,-1.051280,-0.108823,3.230260,-0.783824,-0.799318,-0.823619,-0.634519,...,89462,False,0.0,2017-08-11 13:10:00+00:00,101,False,0.0,2262-04-11 00:00:00+00:00,89462,False
242371,T07,2017-05-02 00:30:00+00:00,0.166353,0.504013,0.335841,-0.529115,-0.825384,-0.860886,-0.857731,-0.638504,...,110,False,0.0,2262-04-11 00:00:00+00:00,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False
329466,T09,2017-05-02 00:30:00+00:00,0.133798,0.531502,0.331753,-0.602091,-0.538037,-0.549877,-0.571722,-0.529612,...,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False,0.0,2262-04-11 00:00:00+00:00,89462,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259722,T07,2017-09-01 00:00:00+00:00,0.194822,0.502250,0.333273,-0.502602,-0.032193,0.012545,0.017976,0.408223,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
346872,T09,2017-09-01 00:00:00+00:00,0.150570,0.544656,0.336676,-0.575309,-0.112670,-0.088962,-0.090070,0.386262,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
172521,T06,2017-09-01 00:00:00+00:00,0.194232,0.513496,0.346333,-0.511821,0.110654,0.094241,0.166022,0.390374,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False
87139,T01,2017-09-01 00:00:00+00:00,0.216726,0.510966,0.353224,-0.499308,-0.078471,-0.040962,-0.015461,0.525644,...,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False,0.0,2262-04-11 00:00:00+00:00,89341,False


Split data into Training and Validation

In [208]:
print(y_test_lead_time)
# FP, FN, TP_lead = 0, 0, []
# warnings = {}
# for i in range(len(y_predict)):
#     if y_test[i] == 0 and y_predict[i] == 1:
#         FP += 1
#     elif y_test[i] == 1:
#         event = Y_event[i]
#         lead_time = Y_lead[i]
#         if event not in warnings and y_pred[i] == 0:
#             warnings[event] = -1
#         elif event not in warnings and y_pred[i] == 1:
#             warnings[event] = lead_time
#         elif event in warnings and warnings[event] < 0:
#             if y_pred[i] == 1:
#                 warnings[event] = lead_time
# for event, lead in warnings.items():
#     if lead < 0:
#         FN += 1
#     else:
#         TP_lead.append(lead)
# 
# FP, FN, TP_lead
# # Calculating saving for hydraulic group
# R, M, I = 20000, 3000, 2000


175254      514
158595       85
419248    89445
216187      228
130033      283
          ...  
167640       21
245829       22
60934     89524
267247    89897
226649      155
Name: Lead_Time_HYDRAULIC_GROUP, Length: 86823, dtype: int64


## Feed into random forest model

Validation Accuracy: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    104263
           1       1.00      1.00      1.00      4266

    accuracy                           1.00    108529
   macro avg       1.00      1.00      1.00    108529
weighted avg       1.00      1.00      1.00    108529
